In [1]:
import time
start = time.time()

import re
import pandas as pd

file = "08 MARKDOWN REPORT.txt"
flag = False
data = []
df = pd.DataFrame(columns=[
    "department",
    "markdown_id", 
    "markdown_description", 
    "date", 
    "style_code", 
    "description", 
    "colour", 
    "price",
    "soh"
])

department = ""
markdown_id = ""
markdown_description = ""
date = ""
style_code = ""
description = ""
time_stamp = ""

with open(file, 'r', encoding='utf-8') as _file:
    for line in _file:
        if line.startswith("Markdown:"):
            x = (line.strip()).split()
            markdown_description = x[3]
            
            if markdown_description.startswith("SC"):
                markdown_id = x[1]
                flag = True
                # print(markdown_id, markdown_description)
            elif markdown_description.startswith("SB"):
                flag = False
                # print(markdown_id, markdown_description)
            elif markdown_description.startswith("JG") or markdown_description.startswith("JAG"):
                flag = False
                # print(markdown_id, markdown_description)
            else:
                markdown_id = x[1]
                flag = True
                # print(markdown_id, markdown_description)
            
        elif flag and line.startswith("Effective"):
            date = (line.strip()).split()[2]
            
        elif flag and (re.match(r"[0-9]{6}E?",line) or line.startswith("SPK")):
            y = (line.strip()).split("  ")
            style_code = y[0].strip()
            description = y[-1].strip()
            
            if style_code.startswith("1"):
                department = "Womenswear"
            elif style_code.startswith("2"):
                department = "Menswear"
            elif style_code.startswith("SPK"):
                department = "Kidswear"
            elif style_code.startswith("5") or style_code.startswith("7"):
                department = "Accessories"
            else:
                department = "Others"
                    
        elif flag and re.match(r"\s{12}[A-Za-z]", line):
            _z = (line.strip()).split("  ")
            z = [i.strip() for i in _z if i]
            soh = int(z[1])
            if soh:
                (colour,price) = (z[0],z[-1])
                data.append((department, markdown_id, markdown_description, date, style_code, description, colour, price,soh))
                # print(data)
                df.loc[len(df)] = (department, markdown_id, markdown_description, date, style_code, description, colour, price,soh)
                # print(data)
            data = []
        
        elif re.match(r"(\d{2}/\d{2}/\d{4} \d{1,2}:\d{2} [AP]M)", line):
            time_stamp = (' '.join(line.split()[:-1]))
            time_stamp = time_stamp.replace("/", "-")
            time_stamp = time_stamp.replace(":", ".")
            print(time_stamp)
            
        else:
            pass

# Step 1: Group by style_code, description, colour
group_cols = ['style_code', 'description', 'colour']

# Step 2: Create price_history as dictionary {price: date}
df['price_history'] = df.apply(lambda row: {row['price']: (row['date'], row['markdown_description'])}, axis=1)

# Step 3: Group and aggregate
def merge_price_history(group):
    # Merge all price_history dicts
    full_history = {}
    for d in group['price_history']:
        full_history.update(d)
    # Return the row with lowest price
    min_row = group.loc[group['price'].astype(float).idxmin()].copy()
    min_row['price_history'] = full_history
    return min_row

deduped_df = df.groupby(group_cols, as_index=False).apply(merge_price_history).reset_index(drop=True)

# Optional: Sort for readability
deduped_df = deduped_df.sort_values(by=['style_code','colour'])

# Save to CSV
deduped_df.to_csv(f"Markdown {time_stamp}.csv", index=False)

end = time.time()
elapsed_s = end - start
print(f"Elapsed time: {elapsed_s:.4f} seconds") # 3.1s - 2.9s


27-03-2025 2.32 PM
Elapsed time: 0.8109 seconds


C:\Users\Siva\AppData\Local\Temp\ipykernel_9832\917087947.py:108: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  deduped_df = df.groupby(group_cols, as_index=False).apply(merge_price_history).reset_index(drop=True)


In [2]:
import time
start = time.perf_counter()

import re
import pandas as pd

# Precompile regexes
date_re  = re.compile(r"(\d{2}/\d{2}/\d{4} \d{1,2}:\d{2} [AP]M)")
style_re = re.compile(r"^(?:[0-9]{6}E?|SPK)")
soh_re   = re.compile(r"^\s{12}[A-Za-z]")

def department_from_style(code):
    if code.startswith("1"):
        return "Womenswear"
    if code.startswith("2"):
        return "Menswear"
    if code.startswith("SPK"):
        return "Kidswear"
    if code[0] in ("5", "7"):
        return "Accessories"
    return "Others"

rows = []
flag = False
current = {}

with open("08 MARKDOWN REPORT.txt", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip("\n")
        if line.startswith("Markdown:"):
            parts = line.split()
            md_desc = parts[3]
            flag = not md_desc.startswith(("SB", "JG", "JAG"))
            if flag:
                current = {
                    "markdown_id": parts[1],
                    "markdown_description": md_desc
                }
        elif flag and line.startswith("Effective"):
            current["date"] = line.split()[2]
        elif flag and style_re.match(line):
            parts = [p.strip() for p in line.split("  ") if p.strip()]
            code = parts[0]
            current.update({
                "style_code": code,
                "description": parts[-1],
                "department": department_from_style(code)
            })
        elif flag and soh_re.match(line):
            parts = [p.strip() for p in line.split("  ") if p.strip()]
            colour, soh, price = parts[0], int(parts[1]), float(parts[-1])
            if soh:
                rows.append({
                    **current,
                    "colour": colour,
                    "soh": soh,
                    "price": price
                })
        elif date_re.match(line):
            ts = line.rsplit(" ", 1)[0]
            time_stamp = ts.replace("/", "-").replace(":", ".")

# Build DataFrame once
df = pd.DataFrame(rows)

# Build price history dict per group
hist = (
    df
    .groupby(["style_code", "description", "colour"])
    .apply(lambda g: {
        row.price: (row.date, row.markdown_description)
        for _, row in g.iterrows()
    })
    .reset_index(name="price_history")
)

# Select the lowest-price row in each group
idx = df.groupby(["style_code", "description", "colour"])["price"].idxmin()
min_rows = df.loc[idx]

# Merge history and sort
deduped = (
    min_rows
    .merge(hist, on=["style_code", "description", "colour"])
    .sort_values(["style_code", "colour"])
)

# print(time_stamp.strip())
deduped.to_csv(f"Markdown {time_stamp.strip()}.csv", index=False)

print(f"Elapsed time: {time.perf_counter() - start:.4f} seconds")


Elapsed time: 0.0948 seconds


C:\Users\Siva\AppData\Local\Temp\ipykernel_9832\3814367164.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: {


In [3]:
import time
from collections import deque
import pandas as pd

start = time.perf_counter()

df_attributes = [
    'sales_person',
    'retail_$_incl_disc',
    'discount',
    'returns',
    'net_$',
    'avg_sale_units',
    'avg_sale_$',
    '%_store_total'
]
df = pd.DataFrame(columns=df_attributes)
buffer = deque()

with open("01 SALES PERFORMANCE REPORT Inc GST calc on Retail.txt", encoding="utf-8") as f:
    # skip header
    for _ in range(9):
        next(f)

    for line in f:
        buffer.append(line)
        if len(buffer) > 6:
            data = buffer.popleft().rstrip("\n").split()

            # if the line split into more than 8 parts, 
            # then the sales_person has spaces
            if len(data) != len(df_attributes):
                # everything except the last 7 tokens is the name
                name = " ".join(data[:-7])
                rest = data[-7:]
                data = [name] + rest

            # now data is exactly 8 items
            row = dict(zip(df_attributes, data))
            df.loc[len(df)] = row

print(df.head())
print(f"Elapsed time: {time.perf_counter() - start:.3f} seconds")


  sales_person                                 retail_$_incl_disc   discount  \
0               ______________________________________________...        NaN   
1    Alejandra                                             134.99       0.00   
2        Alice                                             231.98     -50.00   
3         Amal                                             381.97     -26.00   
4       Amanda                                          57,239.91  10,199.23   

     returns      net_$ avg_sale_units avg_sale_$ %_store_total  
0        NaN        NaN            NaN        NaN           NaN  
1    -134.99       0.00           1.00     134.99         0.00%  
2    -381.97    -149.99           0.67      77.33         0.01%  
3    -237.98     143.99           1.50     190.99         0.01%  
4  -3,265.24  53,974.67           1.67     173.98         1.66%  
Elapsed time: 0.204 seconds
